<a href="https://colab.research.google.com/github/AndresChavez123/Sys2025-2/blob/main/SyS_25_Parcial2S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# instalación de librerías
!pip install streamlit -q

In [16]:
#Se puede usar la API youtube-dl para descargar un video de youtube y extraer el audio en formato mp3.
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz -q

#Libreria para manipulación de archivos de audio
!pip install soundfile -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [17]:
!pip install youtube_dl -q

In [18]:
# cookies
!pip install browser-cookie3

In [19]:
!apt install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [20]:
import gdown
import os


In [21]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [22]:
%%writefile Parcial_2.py

import streamlit as st

st.set_page_config(
    page_title="Parcial 2 - SyS",
    page_icon="📝",
)

# Título principal
st.title("Parcial 2 - Señales y Sistemas")
st.markdown("### Transformada de Fourier y Laplace")

# Portada e información general
st.markdown(
    """
---

#### Universidad Nacional de Colombia – Sede Manizales
**Departamento de Ingeniería Eléctrica, Electrónica y Computación**

**Asignatura:** Señales y Sistemas
**Profesor:** Andres Marino Álvarez Meza, Ph.D.

**Estudiante:** Andres Mauricio Chavez Ceron
**Periodo académico:** 2025-2

---

"""
)

# Descripción de los puntos del parcial
st.markdown(
    """
## Descripción de los puntos

### Punto 1 – Demodulador en amplitud

En este punto se trabaja con un **demodulador en amplitud** presentado en el enunciado del examen.
Asumiendo que el ángulo θ₀ es igual a cero, se debe determinar el **espectro de Fourier teórico** en cada una de las etapas del sistema.

A partir de la simulación de **modulación en amplitud** desarrollada en el Taller 2, y utilizando cinco segundos de una canción tomada de YouTube como señal mensaje, se deben graficar las principales etapas del proceso de **modulación** y **demodulación**, tanto en el **dominio del tiempo** como en el **dominio de la frecuencia**.

En cada una de las etapas se debe reproducir el segmento correspondiente de la canción, de manera que se pueda observar cómo cambia la señal a lo largo del sistema.

> Nota: para la etapa de filtrado pasa bajas, la implementación debe realizarse utilizando la **transformada rápida de Fourier (FFT)**.

---

### Punto 2 – Sistema masa–resorte–amortiguador y modelo eléctrico equivalente

En este punto se analiza un **sistema mecánico masa–resorte–amortiguador**.
Primero, se solicita encontrar la **función de transferencia** que caracteriza el sistema mostrado en la figura del enunciado, asumiendo condiciones iniciales iguales a cero.

Posteriormente, se debe determinar el **sistema eléctrico equivalente** del modelo masa–resorte–amortiguador, a partir del circuito RLC proporcionado. El objetivo es establecer la relación entre los parámetros mecánicos (masa, resorte, amortiguador) y los parámetros eléctricos (resistencia, inductancia y capacitancia), de manera que ambos sistemas presenten una dinámica equivalente de segundo orden.

Finalmente, se deben proponer valores para:

- los parámetros mecánicos: **m**, **k** y **c**, y
- sus equivalentes eléctricos: **R**, **L** y **C**,

con el fin de simular:

- un sistema **subamortiguado**,
- un sistema **sobreamortiguado**, y
- un sistema con **amortiguamiento crítico**.

Para cada uno de estos casos, se debe:

- calcular el **factor de amortiguamiento**,
- la **frecuencia natural amortiguada**,
- la **frecuencia natural no amortiguada**,
- el **tiempo pico**,
- el **tiempo de levantamiento**, y
- el **tiempo de establecimiento**.

Además, para cada condición de amortiguamiento, se deben graficar:

- el **diagrama de polos y ceros**,
- el **diagrama de Bode**,
- la **respuesta al impulso**,
- la **respuesta al escalón**, y
- la **respuesta a la rampa**.

Finalmente, el mismo procedimiento debe repetirse considerando el sistema en **modo lazo cerrado**, comparando los resultados obtenidos con el caso de lazo abierto.

---
"""
)


Overwriting Parcial_2.py


In [23]:
%%writefile Punto_1.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.fft import fft, fftfreq, fftshift
import os
import tempfile
import subprocess
import yt_dlp as youtube_dl       # (yt-dlp ≫ youtube_dl)
import browser_cookie3
from scipy.io import wavfile
from scipy.io.wavfile import write

st.set_page_config(
    page_title="Punto 1 – Demodulación AM",
    page_icon="📡",
    layout="wide"
)

st.title("📡 Punto 1 — Modulación y Demodulación AM")

# ---------------------------------------------------------
#  DESCRIPCIÓN TEÓRICA GENERAL
# ---------------------------------------------------------
st.markdown(
    """
En este panel se implementa un **demodulador en amplitud (AM)** coherente, siguiendo el esquema
del enunciado del parcial.
"""
)

st.markdown("### Modelo de modulación AM")

st.markdown("Dada una **señal mensaje** \\( m(t) \\) y una portadora de amplitud unitaria:")

st.latex(r"c(t) = \cos(\omega_c t) = \cos(2\pi f_c t)")

st.markdown(
    "La **señal modulada en amplitud** (AM de doble banda lateral con portadora, DSB-LC) se expresa como:"
)

st.latex(r"s_{\text{AM}}(t) = \bigl[1 + \mu\, m(t) \bigr]\cos(\omega_c t)")

st.markdown(
    """
donde \\( \\mu \\) es el **índice de modulación**.

En el dominio de la frecuencia, el espectro de Fourier presenta:

- una componente fuerte en la portadora \\( f_c \\),
- y dos **bandas laterales simétricas** alrededor de ella, proporcionales al espectro \\( M(f) \\) del mensaje.
"""
)

st.markdown("### Demodulación coherente con \\( \\theta_0 = 0 \\)")

st.markdown(
    """
Asumiendo que la portadora del demodulador está en fase (\\( \\theta_0 = 0 \\)), el detector productivo
multiplica la señal modulada por la misma portadora:
"""
)

st.latex(r"y(t) = s_{\text{AM}}(t)\,\cos(\omega_c t)")

st.markdown(
    """
Al desarrollar, aparecen términos en baja frecuencia (cerca de DC) y alrededor de \\( 2 f_c \\).
Si a continuación se aplica un **filtro pasa–bajas** con frecuencia de corte mayor que el ancho de banda
del mensaje y menor que \\( 2 f_c \\), se eliminan los componentes de alta frecuencia y queda:
"""
)

st.latex(r"m_{\text{rec}}(t) \approx \tfrac{1}{2}\,m(t)")

st.markdown(
    """
obteniéndose una versión recuperada de la señal mensaje.

En este dashboard se ilustran estas etapas para:

1. Una señal **sintética** tipo pulso rectangular.
2. Un segmento de **audio real** de 5 segundos (canción desde YouTube), tal como pide el parcial.
"""
)

# =========================================================
# 1. EJEMPLO SINTÉTICO: PULSO RECTANGULAR
# =========================================================

st.header("Ejemplo 1 – Pulso rectangular como señal mensaje")

# ---------- Controles en la barra lateral ----------------
sidebar = st.sidebar
sidebar.subheader("⚙️ Parámetros del pulso")
dur = sidebar.slider("Duración del pulso (s)", 0.01, 1.00, 0.10, 0.01)
A   = sidebar.slider("Amplitud del pulso", 0.1,  5.0,  1.0,  0.1)
fc  = sidebar.slider("f\u2095 (Hz) portadora pulso", 10, 500, 100, 10)
mu  = sidebar.slider("Índice de modulación μ (pulso)", 0.1, 1.0, 0.8, 0.05)

# ---------- Señal mensaje m(t) ---------------------------
Fs = 5_000                               # frecuencia de muestreo para el pulso
t  = np.arange(-0.5, 0.5, 1/Fs)

m_t = A * ((t >= -dur/2) & (t <= dur/2)).astype(float)

st.subheader("1️⃣ Mensaje \\( m(t) \\) — Pulso rectangular")
st.markdown(
    f"Se genera un pulso rectangular de amplitud **A = {A}** y duración "
    f"**τ = {dur} s**, centrado en \\( t = 0 \\)."
)

# ---------- Señal AM del pulso ---------------------------
m_norm = m_t / (np.max(np.abs(m_t)) + 1e-9)
s_am = (1 + mu * m_norm) * np.cos(2 * np.pi * fc * t)

st.subheader("2️⃣ Construcción de \\( s_{\\text{AM}}(t) \\) – Pulso modulado")
st.markdown("La señal modulada en amplitud se construye como:")

st.latex(r"s_{\text{AM}}(t) = \bigl[1 + \mu\, m_{\text{norm}}(t)\bigr]\cos(2\pi f_c t)")

st.markdown(
    """
donde \\( m_{\\text{norm}}(t) \\) es una versión normalizada de \\( m(t) \\) y \\( \\mu \\) es el índice de modulación.
"""
)

# ---------- Demodulación coherente del pulso -------------
y_det   = s_am * np.cos(2*np.pi*fc*t)      # detector productivo
fc_lp   = 2 / dur                          # corte ≈ ancho de banda de m(t)
b_lp, a_lp = butter(5, fc_lp, btype='low', fs=Fs)
m_rec   = filtfilt(b_lp, a_lp, y_det)

st.subheader("3️⃣ Demodulación coherente (pulso)")
st.markdown(
    """
Primero se multiplica la señal AM por la misma portadora \\( \\cos(2\\pi f_c t) \\).
Luego se aplica un **filtro pasa–bajas de Butterworth (orden 5)** para eliminar
las componentes cercanas a \\( 2 f_c \\) y recuperar el mensaje original.

> Nota: El uso de un filtro IIR aquí es solo para el **pulso de ejemplo**.
> En la sección de **audio** se implementa el filtro pasa–bajas con **FFT**,
> tal como exige la nota del parcial.
"""
)

# ---------- Espectros del pulso --------------------------
freq_p = fftshift(fftfreq(len(m_t), d=1/Fs))
Mabs_p = fftshift(np.abs(fft(m_t)))
Sabs_p = fftshift(np.abs(fft(s_am)))

# ---------- Gráficas en el tiempo (pulso) ----------------
fig_time, axs = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
axs[0].plot(t, m_t)
axs[0].set_title("m(t) — Pulso rectangular")

axs[1].plot(t, s_am, 'green')
axs[1].set_title("s_AM(t) — Señal modulada en amplitud (pulso)")

axs[2].plot(t, m_rec, label="m_rec(t)")
axs[2].plot(t, m_t, '--', alpha=0.6, label="m(t) original")
axs[2].set_title("Demodulación AM — señal recuperada (pulso)")
axs[2].legend()

for ax in axs:
    ax.grid()
axs[-1].set_xlabel("t (s)")

st.pyplot(fig_time)

# ---------- Gráficas en frecuencia (pulso) ----------------
st.subheader("4️⃣ Espectros del pulso: \\(|M(f)|\\) y \\(|S_{\\text{AM}}(f)|\\)")
fig_freq_p, axfp = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

axfp[0].plot(freq_p, Mabs_p)
axfp[0].set_ylabel(r"$|M(f)|$")
axfp[0].set_title("Espectro del mensaje (pulso)")
axfp[0].grid()

axfp[1].plot(freq_p, Sabs_p)
axfp[1].set_xlabel("Frecuencia (Hz)")
axfp[1].set_ylabel(r"$|S_{\text{AM}}(f)|$")
axfp[1].set_title("Espectro de la señal AM del pulso")
axfp[1].grid()

st.pyplot(fig_freq_p)

# =========================================================
# 2. AUDIO REAL: CANCIÓN DE YOUTUBE (5 s)
# =========================================================

st.header("Ejemplo 2 – Audio real: Modulación y Demodulación AM")

st.markdown(
    """
En esta sección se toma un fragmento de **5 segundos de una canción de YouTube**
(como mensaje \\( m_a(t) \\)) y se implementa:

1. **Modulación AM**.
2. **Demodulación coherente** con portadora en fase (\\( \\theta_0 = 0 \\)).
3. **Filtrado pasa–bajas mediante FFT**, tal como exige la nota del parcial.
4. Análisis en el **tiempo** y en la **frecuencia** de las etapas principales.
"""
)

# ---------- Descarga y conversión a WAV -------------------
YT_URL  = "https://www.youtube.com/watch?v=JtH68PJIQLE"
PREFIX  = "audio_default"

if not os.path.exists(f"{PREFIX}.mp3"):
    st.info("Descargando audio… ")
    try:
        cookies = browser_cookie3.chrome()
    except Exception:
        cookies = None
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": f"{PREFIX}.mp3",
        "quiet": True,
        "no_warnings": True,
    }
    if cookies:
        ydl_opts["cookiejar"] = cookies
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([YT_URL])
        st.success("Descarga completada ✔️")
    except Exception as e:
        st.error(f"No se pudo descargar: {e}")

if os.path.exists(f"{PREFIX}.mp3") and not os.path.exists(f"{PREFIX}.wav"):
    wav_tmp = tempfile.NamedTemporaryFile(suffix=".wav", delete=False).name
    try:
        subprocess.run(
            ["ffmpeg", "-y", "-i", f"{PREFIX}.mp3",
             "-ac", "1", "-ar", "48000", wav_tmp],
            capture_output=True, check=True
        )
        os.replace(wav_tmp, f"{PREFIX}.wav")
    except subprocess.CalledProcessError as err:
        st.error("ffmpeg error:\n" + err.stderr.decode())

# ---------- Cargar y recortar audio -----------------------
if os.path.exists(f"{PREFIX}.wav"):
    Fs_a, audio = wavfile.read(f"{PREFIX}.wav")
    audio = audio.astype(float)
    if audio.ndim == 2:         # estéreo → mono
        audio = audio[:, 0]
    audio /= np.max(np.abs(audio))
    audio = audio[: int(Fs_a * 5)]   # 5 s
    t_a = np.linspace(0, len(audio)/Fs_a, len(audio))

    st.subheader("🎧 Etapa 1 – Mensaje \\( m_a(t) \\) — Audio normalizado (5 s)")
    st.markdown("A continuación se reproduce el segmento de audio utilizado como mensaje:")
    trim_path = tempfile.NamedTemporaryFile(suffix=".wav", delete=False).name
    wavfile.write(trim_path, Fs_a, (audio * 32767).astype(np.int16))
    st.audio(trim_path, format="audio/wav")

# ---------- Parámetros de modulación para el audio --------
sidebar.subheader("⚙️ Parámetros del audio")
fc_a = sidebar.slider("f\u2095 audio (Hz)", 300, 6000, 2000, 100)
mu_a = sidebar.slider("Índice de modulación μ (audio)", 0.1, 1.0, 0.8, 0.05)

if os.path.exists(f"{PREFIX}.wav"):
    # -------- Espectro del mensaje de audio ----------------
    st.subheader("1️⃣ Etapa 1 – Mensaje de audio \\( m_a(t) \\)")

    N_a = len(audio)
    freq_ma = fftshift(fftfreq(N_a, d=1/Fs_a))
    Ma_abs = fftshift(np.abs(fft(audio)))

    fig_msg, (ax_mt, ax_mf) = plt.subplots(2, 1, figsize=(10, 6))
    ax_mt.plot(t_a, audio)
    ax_mt.set_title("Mensaje de audio mₐ(t)")
    ax_mt.set_xlabel("t (s)")
    ax_mt.set_ylabel("Amplitud")
    ax_mt.grid()

    ax_mf.plot(freq_ma, Ma_abs)
    ax_mf.set_title("Espectro |Mₐ(f)| del mensaje de audio")
    ax_mf.set_xlabel("Frecuencia (Hz)")
    ax_mf.set_ylabel("|Mₐ(f)|")
    ax_mf.set_xlim(-10000, 10000)
    ax_mf.grid()

    st.pyplot(fig_msg)

    # -------- Señal AM del audio ---------------------------
    st.subheader("2️⃣ Etapa 2 – Señal modulada en amplitud \\( s_{\\text{AM,audio}}(t) \\)")

    s_am_a = (1 + mu_a * audio) * np.cos(2 * np.pi * fc_a * t_a)

    Sa = fftshift(np.abs(fft(s_am_a)))
    freq_sa = fftshift(fftfreq(N_a, d=1/Fs_a))

    fig_am, (ax_sat, ax_saf) = plt.subplots(2, 1, figsize=(10, 6))
    ax_sat.plot(t_a, s_am_a)
    ax_sat.set_title("s_AM,audio(t) — Señal de audio modulada en AM")
    ax_sat.set_xlabel("t (s)")
    ax_sat.set_ylabel("Amplitud")
    ax_sat.grid()

    ax_saf.plot(freq_sa, Sa)
    ax_saf.set_title("Espectro |S_AM,audio(f)|")
    ax_saf.set_xlabel("Frecuencia (Hz)")
    ax_saf.set_ylabel("|S_AM(f)|")
    ax_saf.set_xlim(-20000, 20000)
    ax_saf.grid()

    st.pyplot(fig_am)

    mod_path = tempfile.NamedTemporaryFile(suffix=".wav", delete=False).name
    wavfile.write(
        mod_path,
        Fs_a,
        (s_am_a / (np.max(np.abs(s_am_a)) + 1e-9) * 32767).astype(np.int16)
    )
    st.markdown("Reproducción opcional de la **señal AM modulada**:")
    st.audio(mod_path, format="audio/wav")

    # -------- Detector productivo y filtrado FFT -----------
    st.subheader("3️⃣ Etapa 3 – Detector productivo y filtrado pasa–bajas (FFT)")

    # Salida del multiplicador (detector productivo)
    y_det_a = s_am_a * np.cos(2 * np.pi * fc_a * t_a)

    # Espectro de y_det_a
    Y_det = fft(y_det_a)
    freq_det = fftfreq(N_a, d=1/Fs_a)

    # Filtro pasa–bajas implementado con FFT
    f_cut = min(5000, fc_a * 0.6)
    mask = np.abs(freq_det) <= f_cut
    Y_lp = Y_det * mask
    rec_a = np.real(np.fft.ifft(Y_lp))

    st.markdown(
        """
La señal modulada se multiplica por \\( \\cos(2\\pi f_c t) \\) (detector productivo).
Luego, la etapa de filtrado pasa–bajas se implementa **en el dominio de la frecuencia**:

1. Se calcula la FFT de la salida del multiplicador.
2. Se aplica una **máscara** que conserva solo las bajas frecuencias (en torno a DC).
3. Se aplica la transformada inversa (IFFT) para volver al dominio del tiempo.

De este modo se cumple la condición del enunciado de implementar el filtro
pasa–bajas utilizando la **transformada rápida de Fourier (FFT)**.
"""
    )

    # -------- Tiempo y frecuencia de la salida del multiplicador ----
    st.subheader("Etapa intermedia – Salida del multiplicador \\( y(t) \\)")

    Y_det_shift = fftshift(np.abs(Y_det))
    freq_det_shift = fftshift(freq_det)

    fig_mult, (ax_yt, ax_yf) = plt.subplots(2, 1, figsize=(10, 6))
    ax_yt.plot(t_a, y_det_a)
    ax_yt.set_title("Salida del multiplicador y(t)")
    ax_yt.set_xlabel("t (s)")
    ax_yt.set_ylabel("Amplitud")
    ax_yt.grid()

    ax_yf.plot(freq_det_shift, Y_det_shift)
    ax_yf.set_title("Espectro de la salida del multiplicador |Y(f)|")
    ax_yf.set_xlabel("Frecuencia (Hz)")
    ax_yf.set_ylabel("|Y(f)|")
    ax_yf.set_xlim(-20000, 20000)
    ax_yf.grid()

    st.pyplot(fig_mult)

    # -------- Señal recuperada y espectro ------------------
    st.subheader("4️⃣ Etapa 4 – Señal recuperada \\( m_{\\text{rec}}(t) \\)")

    R_a = fftshift(np.abs(fft(rec_a)))
    freq_ra = fftshift(fftfreq(N_a, d=1/Fs_a))

    fig_rec, axs_rec = plt.subplots(2, 1, figsize=(10, 6))
    axs_rec[0].plot(t_a, rec_a, label="m_rec(t)")
    axs_rec[0].plot(t_a, audio, '--', alpha=0.6, label="mₐ(t) original")
    axs_rec[0].set_title("Comparación en el tiempo: mensaje original vs recuperado")
    axs_rec[0].set_xlabel("t (s)")
    axs_rec[0].set_ylabel("Amplitud")
    axs_rec[0].legend()
    axs_rec[0].grid()

    axs_rec[1].plot(freq_ra, R_a)
    axs_rec[1].set_title("Espectro de la señal recuperada |M_rec(f)|")
    axs_rec[1].set_xlabel("Frecuencia (Hz)")
    axs_rec[1].set_ylabel("|M_rec(f)|")
    axs_rec[1].set_xlim(-10000, 10000)
    axs_rec[1].grid()

    st.pyplot(fig_rec)

    # -------- Audio demodulado -----------------------------
    rec_a_int16 = np.int16(rec_a / (np.max(np.abs(rec_a)) + 1e-9) * 32767)
    temp_audio = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    write(temp_audio.name, Fs_a, rec_a_int16)

    st.subheader("🔊 Reproducción del audio demodulado")
    st.markdown(
        "Escucha el resultado de la demodulación coherente AM después del filtrado por FFT:"
    )
    st.audio(temp_audio.name, format="audio/wav")


Writing Punto_1.py


In [24]:
!mv Punto_1.py pages/

In [25]:
%%writefile Punto_2.py


import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import TransferFunction, bode, step, impulse, lti

# Función para simular retroalimentación negativa

def feedback(sys1, sys2=1):
    num1, den1 = sys1.num, sys1.den
    if isinstance(sys2, (int, float)):
        num2, den2 = [sys2], [1]
    else:
        num2, den2 = sys2.num, sys2.den

    num_fb = np.polymul(num1, num2)
    den_fb = np.polymul(den1, den2)
    den_total = np.polyadd(den_fb, num_fb)
    return TransferFunction(num1, den_total)

st.set_page_config(
    page_title="Punto 1",
    page_icon="🎯",
    layout="wide"
)

# Título y descripción
st.title("Punto 2: Función de Transferencia y Modelo Eléctrico Equivalente")

st.markdown("""
En este apartado se estudia el comportamiento dinámico de un sistema masa–resorte–amortiguador
y se plantea un modelo eléctrico equivalente que reproduzca su misma estructura de segundo orden.
A partir de la ecuación diferencial del sistema mecánico, se obtiene su representación en el dominio
de Laplace y, posteriormente, se construye un circuito RLC serie que permita analizar un sistema análogo.
""")

# Sección 1: Transformada de Laplace
st.markdown("## Transformada de Laplace del sistema mecánico")

st.markdown("""
Para el sistema masa–resorte–amortiguador, se parte de la ecuación diferencial que relaciona la fuerza aplicada
con el desplazamiento de la masa. Suponiendo condiciones iniciales iguales a cero, se aplica la
**transformada unilateral de Laplace** para pasar al dominio de la frecuencia y simplificar el análisis.
De esta forma se obtiene la siguiente expresión:
""")

st.latex(r"F_E(s) = (m s^2 + c s + k) \cdot Y(s)")

st.markdown("""
A partir de esta relación, se despeja la **función de transferencia en lazo abierto**, la cual establece cómo se
relaciona la salida \\( Y(s) \\) (desplazamiento de la masa) con la entrada \\( F_E(s) \\) (fuerza externa aplicada):
""")

st.latex(r"H(s) = \frac{Y(s)}{F_E(s)} = \frac{1}{m s^2 + c s + k}")

st.markdown("""
Esta función de transferencia corresponde a un sistema de segundo orden caracterizado por los parámetros físicos
\\(m\\), \\(c\\) y \\(k\\), asociados a la masa, el amortiguador y el resorte, respectivamente.
""")

st.markdown("---")

# Sección 2: Modelo Eléctrico Equivalente
st.markdown("## Modelo Eléctrico Equivalente del sistema")

st.markdown("""
Para encontrar un modelo eléctrico equivalente, se considera un circuito **RLC serie** con una entrada de tensión
\\( V_i(t) \\) y una salida \\( V_o(t) \\), medida sobre el **capacitor**. La idea es que el comportamiento dinámico
del circuito refleje la misma forma de ecuación de segundo orden que el sistema mecánico.
En el dominio de Laplace, la relación entre la entrada y la salida del circuito puede escribirse como:
""")

st.latex(r"V_i(s) = \left( L C s^2 + \frac{L}{R} s + 1 \right) V_o(s)")

st.markdown("""
A partir de esta ecuación, se obtiene la **función de transferencia** del circuito eléctrico, que describe cómo
responde el voltaje en el capacitor frente al voltaje aplicado en la entrada:
""")

st.latex(r"\frac{V_o(s)}{V_i(s)} = \frac{1}{L C s^2 + \frac{L}{R} s + 1}")

st.markdown("""
Al comparar esta expresión con la función de transferencia del sistema mecánico, se observa que ambas comparten
una estructura matemática similar de segundo orden. Esto permite asociar convenientemente los parámetros eléctricos
\\(L\\), \\(R\\) y \\(C\\) con las constantes mecánicas \\(m\\), \\(c\\) y \\(k\\), de modo que el circuito pueda utilizarse
como un modelo equivalente del sistema masa–resorte–amortiguador.
""")

st.markdown("---")

# Nota final
st.info(
    "Las dos funciones de transferencia presentan una estructura de segundo orden muy similar, "
    "lo que hace posible utilizar el circuito RLC como modelo eléctrico equivalente del sistema masa–resorte–amortiguador."
)

# =================== Simulación Interactiva =====================
st.markdown("## Simulación Interactiva del Sistema")

# Sidebar: tipo de respuesta y parámetros
st.sidebar.header("Parámetros del Sistema")
tipo_respuesta = st.sidebar.selectbox(
    "Tipo de respuesta a analizar",
    ["Subamortiguada", "Sobreamortiguada", "Amortiguamiento Crítico", "Inestable"]
)

wn = st.sidebar.slider("Frecuencia natural ωₙ (rad/s)", 0.1, 20.0, 5.0)
if tipo_respuesta == "Subamortiguada":
    zeta = st.sidebar.slider("Coeficiente de amortiguamiento ζ", 0.0, 1.0, 0.5)
elif tipo_respuesta == "Sobreamortiguada":
    zeta = st.sidebar.slider("Coeficiente de amortiguamiento ζ", 1.0, 2.0, 1.2)
elif tipo_respuesta == "Amortiguamiento Crítico":
    zeta = 1.0
else:
    zeta = st.sidebar.slider("Coeficiente de amortiguamiento ζ", -1.0, 0.0, -0.1)

# Parámetros físicos y eléctricos
m = 1.0
k = wn**2
c = 2 * zeta * wn * m
L = m
R = c
C = 1 / k

num = [1]
den = [m, c, k]
sys = TransferFunction(num, den)

# Checkbox para seleccionar lazo cerrado
lazo_cerrado = st.sidebar.checkbox("Simular en lazo cerrado", value=False)

# Redefinir sistema si es lazo cerrado
if lazo_cerrado:
    sys = feedback(sys, 1)

# Mostrar parámetros
col1, col2, col3 = st.columns(3)
col1.metric("Masa (m)", f"{m:.2f} kg")
col2.metric("Amortiguador (c)", f"{c:.2f} Ns/m")
col3.metric("Resorte (k)", f"{k:.2f} N/m")

col1, col2, col3 = st.columns(3)
col1.metric("Inductor (L)", f"{L:.2f} H")
col2.metric("Resistor (R)", f"{R:.2f} Ω")
col3.metric("Capacitor (C)", f"{C:.4f} F")

# Respuesta al escalón
t = np.linspace(0, 10, 1000)
t_step, y_step = step(sys, T=t)
st.markdown(f"### Configuración del sistema: {'Lazo cerrado' if lazo_cerrado else 'Lazo abierto'}")

st.markdown("### Respuesta al escalón unitario")
fig1, ax1 = plt.subplots()
ax1.plot(t_step, y_step, label="Escalón")
ax1.set_xlabel("Tiempo (s)")
ax1.set_ylabel("Respuesta")
ax1.grid(True)
st.pyplot(fig1)

# Respuesta al impulso
t_impulse, y_impulse = impulse(sys, T=t)
st.markdown("### Respuesta al impulso")
fig2, ax2 = plt.subplots()
ax2.plot(t_impulse, y_impulse, label="Impulso", color='orange')
ax2.set_xlabel("Tiempo (s)")
ax2.set_ylabel("Respuesta")
ax2.grid(True)
st.pyplot(fig2)

# Respuesta a la rampa
ramp_input = t
_, y_ramp, _ = lti(num, den).output(ramp_input, t)
st.markdown("### Respuesta a una entrada tipo rampa")
fig3, ax3 = plt.subplots()
ax3.plot(t, y_ramp, label="Rampa", color='green')
ax3.set_xlabel("Tiempo (s)")
ax3.set_ylabel("Respuesta")
ax3.grid(True)
st.pyplot(fig3)

# Mapa de polos y ceros
st.markdown("### Mapa de polos y ceros")
poles = np.roots(den)
zeros = np.roots(num)
fig4, ax4 = plt.subplots()
ax4.scatter(np.real(poles), np.imag(poles), color='red', marker='x', label='Polos')
ax4.scatter(np.real(zeros), np.imag(zeros), color='blue', marker='o', label='Ceros')
ax4.axhline(0, color='black', lw=1)
ax4.axvline(0, color='black', lw=1)
ax4.set_xlabel("Parte real")
ax4.set_ylabel("Parte imaginaria")
ax4.grid(True)
ax4.legend()
st.pyplot(fig4)

# Parámetros temporales si subamortiguado
if 0 < zeta < 1:
    wd = wn * np.sqrt(1 - zeta**2)
    tr = np.pi / wd
    tp = np.pi / wd
    Mp = np.exp(-zeta * np.pi / np.sqrt(1 - zeta**2))
    ts = 4 / (zeta * wn)

    st.markdown("### Parámetros temporales del sistema")
    df = pd.DataFrame({
        "Parámetro": [
            "Tiempo de levantamiento (tr)",
            "Tiempo al pico (tp)",
            "Sobreimpulso máximo (Mp)",
            "Tiempo de establecimiento (ts)"
        ],
        "Valor": [
            f"{tr:.3f} s",
            f"{tp:.3f} s",
            f"{Mp*100:.2f} %",
            f"{ts:.3f} s"
        ]
    })
    st.table(df)

st.markdown("### Diagrama de Bode del sistema")
w, mag, phase = bode(sys)
fig5, (ax5, ax6) = plt.subplots(2, 1, figsize=(8, 6))
ax5.semilogx(w, mag)
ax5.set_ylabel("Magnitud (dB)")
ax5.grid(True, which="both")

ax6.semilogx(w, phase)
ax6.set_xlabel("Frecuencia (rad/s)")
ax6.set_ylabel("Fase (°)")
ax6.grid(True, which="both")

st.pyplot(fig5)



Writing Punto_2.py


In [26]:
!mv Punto_2.py pages/

In [27]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run Parcial_2.py &>/content/logs.txt & #Cambiar 0_👋_Hello.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL



--2025-12-03 13:09:47--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-12-03 13:09:47--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-03T14%3A09%3A14Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-03

In [28]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("El proceso de Streamlit ha sido finalizado.")

Digite (1) para finalizar la ejecución del Dashboard: 1
El proceso de Streamlit ha sido finalizado.
